In [ ]:
## Load any changes to local modules
%load_ext autoreload
%autoreload 2

import os
import sys

pwd = %pwd
project_dir = '{0}/../../../'.format(pwd)
module_path = os.path.abspath(os.path.join(project_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import re
import numpy as np
%matplotlib inline
from IPython.display import display

import sqlalchemy
import sqlite3
import struct

from nbcpact import AnalyzeQuantCompare,Peptide,PeptideGroup,UcbreUtils,PeptidesFromPeptideListBuilder
from nbcpact.protdisc import PDReader

In [ ]:
file_path = '/usca/asperapoc/NB-CPACT/NB-CPACT-NIBR/NIBR_loaded_UCB_EN80/Analysis/PD2.1/KEA_isoTOP_DN_All.pdResult'
pdReader = PDReader(pd_result_file=file_path)


pd_target_peptidesDF = pdReader.get_target_peptides()

In [ ]:
ip2_peptide_listDF = pd.read_csv("../../../datanocommit/peptideList.csv")
ip2_peptide_listDF['ip2_sequence'] = ip2_peptide_listDF.sequence

ip2_peptide_listDF['sequence'] = ip2_peptide_listDF.sequence.str.replace('\(\d+\.\d+\)','')

In [ ]:
## Find overlap
pd_target_peptidesDF['in_ip2'] = pd_target_peptidesDF.Sequence.isin(ip2_peptide_listDF['sequence'])
pd_target_peptidesDF['Neglog10_PercolatorqValueMascot'] = pd_target_peptidesDF.PercolatorqValueMascot.apply(lambda x : -np.log10(x+0.0000001))

ip2_peptide_listDF['in_pd'] = ip2_peptide_listDF.sequence.isin(pd_target_peptidesDF.Sequence)

In [ ]:
ip2_peps = set(ip2_peptide_listDF.sequence)
pd_peps = set(pd_target_peptidesDF.Sequence)
intersection = ip2_peps & pd_peps

msg = 'Num IP2 Peps {0}, num PD Peps {1}, num common peps {2}'.format(len(ip2_peps), len(pd_peps), len(intersection))
print(msg)

sns.boxplot(x='in_ip2', y='Neglog10_PercolatorqValueMascot', data=pd_target_peptidesDF)
plt.figure()

sns.violinplot(x="in_ip2", y="Neglog10_PercolatorqValueMascot", data=pd_target_peptidesDF)
plt.figure()